In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

import time
import os
import copy
import math
import glob

from collections import deque
from pathlib import Path

In [2]:
os.getcwd() #↓微妙に違う

'c:\\Users\\tomsh\\OneDrive\\デスクトップ\\研究テーマ\\TCGAPAADdata\\preprocess_data'

In [3]:
path = Path.cwd() #↑微妙に違う

In [4]:
Path(path, "PAAD_Exp.csv")

WindowsPath('c:/Users/tomsh/OneDrive/デスクトップ/研究テーマ/TCGAPAADdata/preprocess_data/PAAD_Exp.csv')

In [5]:
EXP_df = pd.read_csv("C:/Users/tomsh/OneDrive/デスクトップ/研究テーマ/TCGAPAADdata/preprocess_data/PAAD_Exp.csv")

In [6]:
EXP_df.isna().sum().sum()

0

In [7]:
EXP_df.shape
#130人分のデータしかない、、、

(130, 3000)

In [8]:
EXP_df

,CELF3,ADRA1B,SPC24,FIBCD1,TEKT4,MARCO,FFAR2,SLC7A9,FASLG,GPRC5D,...,ACOX2,RAC3,HIATL2,MMP28,RAD51AP1,PLCD3,TOP2A,RIMS3,FKBP9L,NCRNA00085
0,2.736565,1.193008,2.336809,3.255117,1.567721,3.320215,0.631639,2.421343,1.775607,2.402260,...,2.873951,2.865187,2.778039,3.641156,2.893747,3.624310,3.374425,2.630684,2.948463,2.522492
1,3.182000,2.204594,2.415874,1.831403,1.619840,3.282466,2.855049,1.355471,1.971368,2.287101,...,2.885201,2.719042,2.550898,3.372268,2.793141,3.526903,3.488991,2.848533,3.096868,2.906825
2,3.231265,2.515475,2.507271,2.886731,-2.145508,3.451687,2.914352,0.293977,0.293977,2.681636,...,2.726465,3.018371,2.331522,3.610377,2.935887,3.639639,3.558751,3.151823,2.886731,3.001327
3,-0.134657,2.268355,2.367026,3.133404,1.894603,3.237331,2.594578,2.268355,2.028838,2.699508,...,2.861636,2.847244,2.663824,3.519339,3.059547,3.558037,3.544204,2.699508,3.034963,2.639873
4,2.915137,2.619283,2.228052,2.549654,0.137550,3.158111,2.094228,1.424921,2.188869,2.418993,...,2.853863,2.683018,2.228052,3.495372,2.895283,3.527122,3.429515,2.888272,3.292343,2.873591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,3.038540,2.603369,2.330913,2.477391,2.256251,2.685849,0.107161,2.160066,2.303058,2.112690,...,2.704869,3.004575,2.673743,3.280016,3.031232,3.424883,3.536979,3.067876,3.027483,3.044738
126,3.141908,1.613815,1.504989,3.248107,1.504989,2.514170,2.324277,1.364666,2.654814,2.256050,...,2.973275,2.931361,2.353673,3.361543,2.773621,3.513814,3.223943,2.937843,2.692139,3.038970
127,2.597518,2.417896,1.836454,1.658234,2.377349,3.708627,2.330774,1.206832,2.130495,1.206832,...,2.975057,2.963500,2.445142,3.481236,2.747232,3.489073,3.291381,2.673257,2.837182,2.973928
128,2.672047,2.792286,2.138718,3.336379,1.897485,2.826775,1.712888,1.897485,1.495044,2.675498,...,2.485452,2.710759,2.306852,3.544067,2.955197,3.692108,3.394474,2.813153,2.749669,3.065801


In [9]:
class AEDataset():
    def __init__(self, X, y):
        self.X = torch.Tensor(X.values)
        self.y = torch.Tensor(y.values)

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx] 
        #後でdata_loader作るときに、シャッフルした方を"train"にして、シャッフルしてないほうを"test"にして辞書にする
        #キーからデータを取ってこれるようにgetitem関数作る

Auto Encoder

In [10]:
class Encoder(nn.Module):
    def __init__(self, input_features, encoding_dim): #encoding_dimはリストで渡す
        super().__init__()
        self.fc1 = nn.Linear(input_features, encoding_dim[0])
        self.pool1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(encoding_dim[0], encoding_dim[1])
        self.pool2 = nn.Dropout(0.5)
        print

    def forward(self, x):
        #print("Encoder入った")
        #print("Encoder入力のxのsizeは:{}".format(x.shape))
        x = torch.tanh(self.fc1(x)) #ReLUだったら入力負の時0になっちゃうからいったんtanhでやってみる。元の論文の原因これReLUにしてたからでは？
        x = self.pool1(x)
        x = torch.tanh(self.fc2(x))
        x = self.pool2(x)
        #print("Encoder出力のxのsizeは:{}".format(x.shape))
        #print("Encoder出た")
        return x
    
class Decoder(nn.Module):
    def __init__(self, encoding_dim, input_features):
        super().__init__()
        self.fc3 = nn.Linear(encoding_dim[1], encoding_dim[0])
        self.fc4 = nn.Linear(encoding_dim[0], input_features)

    def forward(self, x):
        x = torch.tanh(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        return x
    
class AutoEncoder(nn.Module):
    def __init__(self, input_features, encoding_dim):
        super().__init__()
        self.encoder = Encoder(input_features, encoding_dim)
        self.decoder = Decoder(encoding_dim, input_features)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

Train関数

In [11]:
def train_model(model, loss_func, optimizer, data_loader, n_epochs, fout, device):
    #初期化
    pkl_queue = deque()
    best_loss = 100.0
    best_epoch = 0
    best_model_weights = model.state_dict()  # state_dict()はtorchの関数
    since = time.time()
    end = time.time()

    print(model, "\n")

    for epoch in range(n_epochs):
        print("EPOCH:{}/{}".format(epoch+1, n_epochs), end="")
        print("EPOCH:{}/{}".format(epoch+1, n_epochs), end="", file=fout)

        for phase in ["train"]:
            model.train(True)

            #データの指定
            data = data_loader[phase]

            #初期化
            running_loss = 0

            #ミニバッチに対するループ処理
            for _, (data_train, target_train) in enumerate(data):
                optimizer.zero_grad()
                x = data_train.to(device)
                y = target_train.to(device)

                with torch.set_grad_enabled(phase == "train"):
                    y_pred = model(x)
                    loss = loss_func(y_pred, y)

                    loss.backward()
                    optimizer.step()

                running_loss += loss.item()
                
            epoch_loss = running_loss / (len(data)/len(x))

            #最も損失が小さかったモデルを保存
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                best_epoch = epoch
                best_model_weights = copy.deepcopy(model.state_dict())
                torch.save(best_model_weights, "{}_epoch{}.pkl".format(fout.name.split(".txt")[0], epoch+1))
                pkl_queue.append("{}_epoch{}.pkl".format(fout.name.split(".txt")[0], epoch+1))

                if len(pkl_queue) > 1:
                    pkl_file = pkl_queue.popleft()
                    os.remove(pkl_file)
            
            #予測の出力
            print(", {}Loss:{:.4f}, Time:{:.4f}".format(phase, epoch_loss, time.time()-end), end="")
            print(", {}Loss:{:.4f}, Time:{:.4f}".format(phase, epoch_loss, time.time()-end), end="", file=fout)
            print("\n", end="")
            print("\n", end="", file=fout)

            end = time.time()

    #トレーニング結果の表示
    time_elapsed = time.time() - since
    print("\nTraining completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60))
    print("Best loss: {:.4f} at epoch {}".format(best_loss, best_epoch))

EPOCH-Lossグラフの描画

In [68]:
def plot_loss(file, name):

    #トレーニングログファイルの読み込み
    df = pd.read_csv(file, header=None, sep=r"\s+")
    for col in df.columns:
        df[col] = df[col].str.replace('[A-Za-z]+:', '', regex=True)
        df[col] = df[col].str.replace(',', '', regex=True)    

    #線グラフを作成
    fig, ax = plt.subplots()
    plt.plot(range(len(df)), df.iloc[:, 1].astype('float'))
    ax.set_title(f"MSE loss for \n{file}")
    ax.set_xlabel("EPOCHS")
    ax.set_ylabel("MSE loss")
    fig.savefig(f"{name}.png")
    plt.close(fig)

In [67]:
"""
df_temp = pd.read_csv(str(Path(path, 'AutoEncoder', 'AE_lr8e-05_epochs20_batch1.txt')), header=None, sep=r"\s+")
df_temp.columns = ['Epoch', 'Loss', 'Time']
df_temp
for col in df_temp.columns:
    df_temp[col] = df_temp[col].str.replace('[A-Za-z]+:', '', regex=True)
    df_temp[col] = df_temp[col].str.replace(',', '', regex=True)
#df_temp['Epoch'].str.replace('[A-Za-z]+:', '', regex=True)
df_temp['Loss'].astype('float')
"""


0     7.0651
1     5.0733
2     4.1891
3     3.9501
4     3.7942
5     3.7198
6     3.6719
7     3.6438
8     3.6235
9     3.6026
10    3.5959
11    3.5841
12    3.5834
13    3.5829
14    3.5706
15    3.5682
16    3.5639
17    3.5593
18    3.5637
19    3.5599
Name: Loss, dtype: float64

モデルの評価

In [13]:
def eval_model(model, data_loader, device):
    #初期化
    running_mse = 0
    preds = []

    data = data_loader["test"]
    model.eval()

    for data_test, target_test in data:
        x = data_test.to(device)
        y = target_test.to(device)

        #予測
        with torch.no_grad():
            y_pred = model(x)

            #MSEの計算
            sq_loss = ((y_pred - y)*(y_pred - y)).sum().data
            running_mse += sq_loss

            preds.append(y_pred[0])
    #予測スコアを表示
    preds = np.vstack(preds)
    mse = math.sqrt(running_mse / len(data))
    print("MSE: {}".format(mse))

    return preds



パラメータの指定

In [82]:
batch_size = 1
n_epochs = 30
lr = 0.000006

fileがなかった時に新しくそのディレクトリにfileを作る関数

In [15]:
def check_path(filename):
    if not Path(filename).is_dir():
        Path(filename).parents[0].mkdir(parents=True, exist_ok=True)
    return filename

実行する関数

In [83]:
def main():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    path = Path.cwd()

    #input_dataの読み込み
    df = pd.read_csv(Path(path, "PAAD_Exp.csv"))
    X = df.astype(np.float32)

    #load AutoEncoder
    model = AutoEncoder(3000, [200, 10])
    model = model.to(device)

    #dataset 作る
    AEdata = AEDataset(X, X)
    train_data = DataLoader(AEdata, batch_size=batch_size, shuffle=True)
    test_data = DataLoader(AEdata, batch_size=batch_size, shuffle=False)
    data_loader = {"train": train_data, "test" : test_data}

    loss_func = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    #トレーニングログファイルの設定
    train_log = "AE_lr{}_epochs{}_batch{}.txt".format(lr, n_epochs, batch_size)
    fout = open(check_path(str(Path(path, "AutoEncoder", train_log))), "w") #ログファイルを書き込みモードで開く

    #train AutoEncoder
    train_model(model, loss_func, optimizer, data_loader, n_epochs, fout, device)

    fout.close() #書き込み終了

    #train lossの書き出し
    plot_loss(str(Path(path, "AutoEncoder", train_log)), check_path(str(Path(path, "AutoEncoder", f"epoch{n_epochs}_loss"))))

    #最も性能の良いAEモデルの読み込み
    trained_model = glob.glob(str(Path(path, "AutoEncoder", "{}_*.pkl".format(train_log.split(".txt")[0]))))[0]
    model.load_state_dict(torch.load(trained_model), strict=False)

    #学習済みモデルを用いた予測
    decoded_result = eval_model(model, data_loader, device)
    print(decoded_result)

    #もっともよい学習済みモデルからの特徴抽出
    bottleneck_features = model.encoder(torch.Tensor(X.values)).detach().numpy()
    print(bottleneck_features)

    #低次元特徴量の保存
    np.savetxt(check_path(str(Path(path, "Bottleneck", f"epoch{n_epochs}_std_Exp.csv"))), bottleneck_features, delimiter=",")
    np.save(check_path(str(Path(path, "Bottleneck", f"epoch{n_epochs}_std_Exp.npy"))), bottleneck_features)

    #decodeしたデータの保存
    np.savetxt(check_path(str(Path(path, "DecodedExp", f"epoch{n_epochs}_std_Exp.csv"))), decoded_result, delimiter=",")


if __name__ == "__main__":
    main()

    

AutoEncoder(
  (encoder): Encoder(
    (fc1): Linear(in_features=3000, out_features=200, bias=True)
    (pool1): Dropout(p=0.5, inplace=False)
    (fc2): Linear(in_features=200, out_features=10, bias=True)
    (pool2): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (fc3): Linear(in_features=10, out_features=200, bias=True)
    (fc4): Linear(in_features=200, out_features=3000, bias=True)
  )
) 

EPOCH:1/30, trainLoss:8.0845, Time:3.0073
EPOCH:2/30, trainLoss:7.8724, Time:2.7945
EPOCH:3/30, trainLoss:7.6398, Time:2.6150
EPOCH:4/30, trainLoss:7.4287, Time:1.7154
EPOCH:5/30, trainLoss:7.1911, Time:1.7403
EPOCH:6/30, trainLoss:7.0279, Time:1.8954
EPOCH:7/30, trainLoss:6.8097, Time:1.8986
EPOCH:8/30, trainLoss:6.6263, Time:2.0944
EPOCH:9/30, trainLoss:6.3969, Time:1.8039
EPOCH:10/30, trainLoss:6.2471, Time:1.9358
EPOCH:11/30, trainLoss:6.0607, Time:2.8854
EPOCH:12/30, trainLoss:5.9337, Time:2.1424
EPOCH:13/30, trainLoss:5.7240, Time:2.0108
EPOCH:14/30, trainLoss:5.6028, Time:1.7

C:\Users\tomsh\AppData\Local\Temp\ipykernel_30352\1863318446.py:15: UserWarning: Glyph 12487 (\N{KATAKANA LETTER DE}) missing from current font.
  fig.savefig(f"{name}.png")
C:\Users\tomsh\AppData\Local\Temp\ipykernel_30352\1863318446.py:15: UserWarning: Glyph 12473 (\N{KATAKANA LETTER SU}) missing from current font.
  fig.savefig(f"{name}.png")
C:\Users\tomsh\AppData\Local\Temp\ipykernel_30352\1863318446.py:15: UserWarning: Glyph 12463 (\N{KATAKANA LETTER KU}) missing from current font.
  fig.savefig(f"{name}.png")
C:\Users\tomsh\AppData\Local\Temp\ipykernel_30352\1863318446.py:15: UserWarning: Glyph 12488 (\N{KATAKANA LETTER TO}) missing from current font.
  fig.savefig(f"{name}.png")
C:\Users\tomsh\AppData\Local\Temp\ipykernel_30352\1863318446.py:15: UserWarning: Glyph 12483 (\N{KATAKANA LETTER SMALL TU}) missing from current font.
  fig.savefig(f"{name}.png")
C:\Users\tomsh\AppData\Local\Temp\ipykernel_30352\1863318446.py:15: UserWarning: Glyph 12503 (\N{KATAKANA LETTER PU}) missin

MSE: 108.80584054865805
[[0.8751121  0.84042287 0.8568561  ... 0.78213936 0.8633665  0.8928007 ]
 [0.87512285 0.84043795 0.85685384 ... 0.7821438  0.8633786  0.8928021 ]
 [0.87513167 0.84043646 0.85682744 ... 0.78213274 0.86338323 0.892797  ]
 ...
 [0.875128   0.8404415  0.8568415  ... 0.78214085 0.8633823  0.8927984 ]
 [0.87512946 0.84044254 0.8568485  ... 0.7821392  0.8633458  0.89279187]
 [0.8751624  0.8404688  0.8568187  ... 0.7821458  0.86338115 0.89279985]]
[[ 0.99543417  0.9920536   0.99684364 ... -0.99696326 -0.99645925
  -0.9925035 ]
 [ 0.99543864  0.99198276  0.9968214  ... -0.99693364 -0.9964385
  -0.99244887]
 [ 0.99554425  0.99182874  0.9966045  ... -0.997023   -0.99642766
  -0.9926946 ]
 ...
 [ 0.99553496  0.991913    0.99661726 ... -0.9970293  -0.99643904
  -0.9926074 ]
 [ 0.9951535   0.99243635  0.99682254 ... -0.99675065 -0.9962982
  -0.9922765 ]
 [ 0.99533224  0.99195254  0.99659693 ... -0.9969098  -0.9963631
  -0.99257326]]


In [17]:
"""
class LinearReLU(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearReLU, self).__init__()
        self.f = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(True)
        )
    
    def forward(self, x):
        h = self,f(x)
        return(h)
    
class Encoder(nn.Module):
    def __init__(self, bottle_dim)
"""

'\nclass LinearReLU(nn.Module):\n    def __init__(self, input_dim, output_dim):\n        super(LinearReLU, self).__init__()\n        self.f = nn.Sequential(\n            nn.Linear(input_dim, output_dim),\n            nn.BatchNorm1d(output_dim),\n            nn.ReLU(True)\n        )\n    \n    def forward(self, x):\n        h = self,f(x)\n        return(h)\n    \nclass Encoder(nn.Module):\n    def __init__(self, bottle_dim)\n'